**Monthly Churn Preparation**

**Created by Tom Tibbett**

**Description:** The following code is taken from munging a real data set with monthly recurring data.  The original usage was preparation for a Churn Model.  Months where the individual did not participate were blank, so no one column had their most recent device. Passwords and other things have been removed, so no actual data will be called.  However, this code can be adapted to do the following things:

* Call a table in a HANA SQL database and insert it (wholly or in part) into a Pandas DataFrame
* Create a function that identifies the first non-null value outside of the ID
* Counts the non-null fields (excluding ID), implying how many months they participated in the service.
* Subsets based on whether the first non-null value differed from the last non-null value

*Note:* In this database, the columns progress in reverse order.  In this example, June2016 is column number 2, while May2016 is column number 3, etc.

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import pyhdb

In [ ]:
# Connecting to HANA
connection = pyhdb.connect(
    host="host.com",
    port=30015,
    user="user",
    password="pass"
)

cursor = connection.cursor()
cursor.execute("""INSERT YOUR SQL QUERY HERE""")
df= pd.DataFrame(cursor.fetchall())

In [ ]:
# Giving Customized Column Names, based on how many columns you call
df.columns=['ID', 'June2016', 'May2016', 'April2016', 'March2016', 'Feb2016', 'Jan2016', 'Dec2015', 'Nov2015',
           'Oct2015', 'Sept2015', 'Aug2015', 'July2015', 'June2015', 'May2015']

In [ ]:
# Grabs the First Non-Null Column
def func(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

df['MonthsActive']=np.sum(df.notnull(), axis=1)-1 # Accounts for the Unique ID Field

df['Recent']=df.iloc[:,1:].apply(func, axis=1) # You want to skip ID, so .iloc is necessary

In [ ]:
# Creates and populates a new column based on whether the first non-null and last date column match. 
df.loc[df['Recent']==df['May2015'], 'Upgrade']='No Change' # All inputs here are categorical
df.loc[df['Recent']!=df['May2015'], 'Upgrade']='Change'

In [ ]:
# Creates a summary table of constituents by number of months they were non-null.
print 'All Constituents Counts of Monthly Activity'
MA=pd.DataFrame(df['MonthsActive'].value_counts().reset_index())
MA.columns= ['MonthsActive', 'Cons_Counts']
MA=MA.sort_values('MonthsActive')
MA['Percent']=MA['Cons_Counts']/np.sum(MA['Cons_Counts'], axis=0)
print MA.reset_index(drop='index')

# Prints an overall total of participants
print '\nThere are', np.sum(MA['Cons_Counts']), 'constituents in all.'

In [ ]:
# Individual table of those whom changed
print '\nMonthly Participation Among Those Whom Changed'
print df[['ID','May2015','Recent', 'MonthsActive']].ix[df['Upgrade']=='Upgrade'].sort_values('MonthsActive')

In [ ]:
# Creates a summary table of those whom changed
print 'Counts by Month Among Those Whom Changed\n'
MAU=pd.DataFrame(df['MonthsActive'].ix[df['Upgrade']=='Upgrade'].value_counts().reset_index())
MAU.columns= ['MonthsActive', 'Cons_Counts']
MAU=MAU.sort_values('MonthsActive')
MAU['Percent']=MAU['Cons_Counts']/np.sum(MAU['Cons_Counts'], axis=0)
print MAU.reset_index(drop='index')
print '\n'
print 'There are', np.sum(MAU['Cons_Counts']), 'constituents in all whom changed.'